## Meta genomics DWV logboek

Bezig geweest met dorado en pod5 werkende krijgen.  

Pod5 format werkende gekregen. 

## Opzet
De data voor dit onderzoek is verkregen op 5 november 2019, en er is een eerder kort onderzoek op 5 and 18 december 2019 naar deze data gedaan. Uit het eerdere onderzoek is gebleken dat de genoomsequentie van DWV 11kb lang is. De data gemaakt tijdens het vorige onderzoek wordt in dit logboek vergeleken met onze eigen data.

### Benodigdheden
Alle door ons gebruikte libraries en dependencies staan in het bijgevoegde environments.yaml, Hier kan een conda environment van gemaakt worden met het volgende commando:
> $ conda env create --name [envname] --file=environments.yml
>
Deze conda environment kan worden geactiveerd met het volgende commando:
> $ conda activate [envname]
>
Vervang in deze commando's '[envname]' met een omgevingsnaam naar keuze. \
Alle libraries/dependencies hebben wij met behulp van pip install of conda install in onze conda environment geïnstalleerd.

Door ons gebruikte tools:
- Awk (5.2.1)
- Clustal Omega (1.2.4)


Naast guppy, die op het graphix1 systeem is gerunt, zijn alle tools op het assemblix systeem (Intel® Xeon® Gold 6248 Processor, Integrated Matrox G200eW3, 881GB MEM) gerunt.

## Basecalling
Basecalling is het vertalen van de elektrische stroomveranderingen die waren gemeten toen de nucleotiden door de Nanopore gingen, naar nucleobasen (een A,T,C of G). Voor deze vertaling zijn er meerdere tools om uit te kiezen. De snelste en best presterende basecaller is Dorado, dus we begonnen met deze. \
Na alles te installeren en het te runnen kregen we deze error: 

> No supported chemistry for Flowcell:'FLO-MIN106', Kit:'SQK-RNA001' and Sampling Rate:3012 

Onze data is met een FLO-MIN106 Flowcell gebasecalled.
Volgens een github comment van raronan op de dorado pagina: https://github.com/nanoporetech/dorado/issues/214 Is onze pore versie r9.4.1. 

Het lijkt erop dat dorado ons type flowcell niet ondersteunt. Als we opzoeken of dorado specifiek deze flowcell ondersteunt kunnen we niet direct wat over ons type vinden. Maar wel een kort lijstje aan versies die niet meer ondersteund worden, deze lijst https://github.com/nanoporetech/dorado/issues/650 gaat van r9.5 tot r10.4, onze versie (r0.4.1) is ouder dan dit. Het lijkt erop dat wij niet dorado kunnen gebruiken voor onze data. 

We hebben besloten de basecalling met guppy te doen, deze tool werkt met oude(re) flowcells. Guppy is echter wel minder snel dan Dorado.

### Guppy

Guppy is een basecaller die sinds eind 2017 openbaar beschikbaar is. \
Wij hebben guppy gebruikt om de fast5 data opnieuw te basecallen. Hiervoor hebben wij deze rule gebruikt:


In [ ]:
rule guppy: 

input: 

'/commons/Themas/Thema07/metagenomics/DWV/Sick_Beezzz/20191218_1217_MN30556_FAL66274_dd5dacb8/fast5_pass/' 

output: 

'/students/2023-2024/Thema07/Sick_beez/guppy_out/' 

shell: 

'/opt/ont/ont-guppy/bin/guppy_basecaller -i {input} -s {output} -c /opt/ont/ont-guppy/data/rna_r9.4.1_70bps_hac.cfg --min_qscore 7 --recursive -x 'cuda:0' --num_callers 4 --gpu_runners_per_device 4 --chunks_per_runner 1024 --chunk_size 1000 --compress_fastq' 

 

Wij hebben een nieuwere model gebruikt zodat guppy accurater de bases uit de fast5 files kan callen. Wij hebben dit op het graphix1 systeem gerunt op een NVIDIA GeForce GTX 1080 Ti. Het basecallen van alle data duurde ongeveer 15 minuten. \
Basecallen werd vroeger op uitsluitend CPU (processor) gedaan, maar tegenwoordig wordt er meestal een CUDA enabled GPU gebruikt. Op deze pagina van Nvidia staan alle CUDA Enabled GPU's: https://developer.nvidia.com/cuda-gpus. CUDA is eigendom van Nvidia, en dus de enige producent.

In de home stonden de conda channels, hier stond een URL bij, deze hebben wij verwijderd zodat we weer met guppy kunnen basecallen.  

## Quality control
Quility control helpt bij het snel identificeren van reads van slechte kwaliteit, het vinden/aantonen van problemen met de gegevens en geeft algemene informatie over de gegeven sample.
Voor de quality control twijfelden we tussen de tools: FastQC en NanoPlot. We hadden meer ervaring met fastqc maar NanoPlot is specifiek voor MinION/Nanopore data gemaakt. Na met beide kort een testrun te doen zijn we op NanoPlot uitgekomen. NanoPlot is makkelijker in gebruik voor MinION/Nanopore data.

### NanoPlot op de oude data
De door ons gebruikte Nanopore data is al eerder gebasecalled in december 2016 met guppy versie 3.2.6+afc8e14. Om te beginnen gingen we de kwaliteit van deze oude data kijken. \

We hebben NanoPlot geprobeerd te runnen op de oude data. Het was niet mogelijk om NanoPlot op onze folder met guppy data te runnen, dus hebben we NanoPlot op 1 bestand toegepast.

In [ ]:
rule nanoplot: 
    input: 
        reads="/homes/haroorda/periode_7/sick_beez/zipje/your_archive.tar.gz" 
    output: 
        html="results_2/nanoplot_reads.html", 
    shell: 
        """ 
        mkdir -p temp_directory && \ 
        tar -zxvf {input.reads} -C temp_directory && \ 
        NanoPlot --fastq temp_directory/*.fastq -o {output.html} && \ 
        rm -rf temp_directory 
        """ 

Dit is niet gelukt, we kregen errors die niet op te lossen waren. Dit kwam omdat we het op de hele folder met de fastq files wilden runen.
Volgens de documentatie had dit moeten kunnen op de fastq files. 
Wel is het gelukt om NanoPlot te runnen op 1 file. 

In [ ]:
/commons/docent/Thema07/metagenomics/DWV/Sick_Beezzz/20191218_1217_MN30556_FAL66274_dd5dacb8/guppy_out_fast/pass$ NanoPlot --fastq fastq_runid_61214d90674fb476eeea0df13759443ace338524_0_0.fastq -o ~/Desktop/test_nanoplot 

### NanoPlot op de nieuwe data
We hebben NanoPlot op de nieuwe versie van hetzelfde bestand toegepast. NanoPlot op een hele map te toepassen werkt vooralsnog niet. We hebben de NanoPlots van de oude en nieuwe gebasecallede data vergeleken en we zagen dat de gemiddelde quality van 8.5 naar 9.9 ging.

Op alle de data die uit onze guppy run kwam hebben wij NanoPlot gerunt. Omdat dit niet op de hele folder kan, hebben we het per bestand gedaan. Hiervoor hebben wij deze code gebruikt:

In [ ]:
input_folder="/students/2023-2024/Thema07/Sick_beez/guppy_out/pass/" 

counter=0 

for file in "$input_folder"*.fastq.gz; do 

counter_str=$(printf "%04d" $counter) 

NanoPlot --fastq "$file" --outdir /students/2023-2024/Thema07/Sick_beez/Nanoplot/ --prefix "$counter_str" --only-report 

((counter++)) 

done 

Omdat we het per bestand runnen, eindigen we met 60 losse NanoPlot html-bestanden. We moeten eerst de fastq files die uit guppy komen samenvoegen tot 1 groot fastq bestand. Wat we al wel konden zien in de 60 gemaakte NanoPlot bestanden was dat er reads waren die tussen de 10 en 11 duizend baseparen zaten. Dit komt overeen met de lengte van het DWV genoom dat in het vorige onderzoek was aangetoond. Als wij een bestand maken waar alle reads in staan kunnen wij daarna alle reads boven de 10k baseparen selecteren om alleen de DWV genomen op te nemen.

### Cat command
Hieronder worden de files geconcataneerd met behulp van het cat commando. Het is niet heel efficient, maar het werkt wel. FASTQ en FASTA bestanden bestaan alleen uit tekst, de stukken tekst van meerdere bestanden kunnen zonder problemen in 1 bestand achter elkaar worden geplaatst. \
Als input pakken we alle bestanden uit '/students/2023-2024/Thema07/Sick_beez/guppy_out/pass/' die eindigen op '.fastq.gz', deze bestanden unzippen we met gunzip, dat voegen we dan aan elkaar met het cat commando. Tot slot zippen we het bestand met gzip. De output is 1 grote (.fastq.fz) file met alle fastq files erin verwerkt.

In [ ]:
cat /students/2023-2024/Thema07/Sick_beez/guppy_out/pass/*.fastq.gz | gunzip -c > /students/2023-2024/Thema07/Sick_beez/big_guppy/big_guppy.fastq 

gzip /students/2023-2024/Thema07/Sick_beez/big_guppy/big_guppy.fastq

Nu runnen we NanoPlot op het geconcateneerde fastq bestand. Er blijft een html bestand over.

In [ ]:
NanoPlot --fastq "/students/2023-2024/Thema07/Sick_beez/big_guppy/big_guppy.fastq.gz" --outdir /students/2023-2024/Thema07/Sick_beez/big_nanoplot/ --prefix "big_" --only-report 

### Chopper 
<a href="(https://github.com/wdecoster/chopper)">Chopper</a>
is een combinatie van de tools: NanoFilt en NanoLyse. Deze tool filtert en trimt een fastq-bestand. Op basis van gemiddelde read quality en minimale of maximale read length. Ook kan de tool een bepaald aantal baseparen van het begin of het einde van de reads verwijderen. Dit kan worden gedaan om adapter sequenties, Primer Artifacts of baseparen met slechte kwaliteit te verwijderen (wat vaker voorkomt aan het einde van een read) eruit te halen, en zo de kwaliteit van de data te verhogen.

DWV heeft ongeveer een lengte van 10kb - 11kb dus om deze eruit te halen hebben we alle reads boven de 10kb uit het grote bestand gefilterd om dit te doen hebben wij chopper gebruikt. Hiervoor is deze code gebruikt:

In [ ]:
gunzip -c /students/2023-2024/Thema07/Sick_beez/big_guppy/big_guppy.fastq.gz | chopper -l 10000 | gzip > /students/2023-2024/Thema07/Sick_beez/trimmed_fastq/filtered_reads.fastq.gz 

Wij hebben chopper alleen gebruikt voor de sequentie lengte omdat er bij het basecallen al op een minimum quality van 7 getrimmed is, de kwaliteit was al in orde.

Hierna hebben wij NanoPlot op de getrimde data gerunt:

In [ ]:
NanoPlot --fastq "/students/2023-2024/Thema07/Sick_beez/trimmed_fastq/filtered_reads.fastq.gz" --outdir /students/2023-2024/Thema07/Sick_beez/trimmed_nanoplot/ --prefix "trimmed_" --only-report 

### FASTQ naar FASTA
Daarna hebben we in AWK de FASTQ file omgezet naar een FASTA file om deze te gebruiken met de volgende tool, Clustal Omega.
AWK is een populaire text-processing programmeertaal op GNU/Linux. \
Het van FASTQ omzetten in FASTA lukte aanvankelijk niet in snakemake en dit hadden we dus met terminal code gedaan. Uiteindelijk hebben we het wel werkende gekregen in snakemake, dit hebben we gedaan met het volgende commando:

In [ ]:
gunzip -c /students/2023-2024/Thema07/Sick_beez/trimmed_fastq/filtered_reads.fastq.gz | awk '{if(NR%4==1) {printf(">%s\n",substr($0,2));} else if(NR%4==2) print;}' > /students/2023-2024/Thema07/Sick_beez/trimmed_fasta/filtered_reads.fasta 

## Clustal Omega
Clustal Omega is een multiple sequence alignment programma. Net als de voorganger ClustalW, kan het DNA-, RNA- en eiwitsequenties allignen. ClustalW komt uit 1994, en Clustal Omega komt uit 2016.

Over het paasweekend (28 maart – 2 april) hebben wij clustal omega aangezet met 1 thread. Na het paasweekend was deze nog niet klaar. We hebben clustal omega opnieuw aangezet met dit keer 40 threads. 

We hebben op 1 thread - 30 uur niet klaar  
Zonder guide tree op 40 threads - 3 uur 
Met guide tree op 80 threads – 2 uur 

In [ ]:
clustalo -i /students/2023-2024/Thema07/Sick_beez/trimmed_fasta/fileter_reads.fasta -o /students/2023-2024/Thema07/Sick_beez/output_clustalo/output_omega.msa.fa


Toen hebben ervoor gekozen om deze command line nog een keer te runnen maar dan met 40 threads in plaats van de default threads, wat 1 bleek te zijn.

In [ ]:
clustalo -i /students/2023-2024/Thema07/Sick_beez/trimmed_fasta/filtered_reads.fasta -o /students/2023-2024/Thema07/Sick_beez/output_clustalo/output_omega.msa.fa --threads 40


Hij deed hier nu maar 3-4 uur over, dit is een hele verbetering. Echter hadden we geen guide tree optie meegegeven, waardoor we het opnieuw moesten runnen.

In [ ]:
clustalo -i /students/2023-2024/Thema07/Sick_beez/trimmed_fasta/filtered_reads.fasta -o /students/2023-2024/Thema07/Sick_beez/output_clustalo/output_omega.aln --threads 80 --guidetree-out /students/2023-2024/Thema07/Sick_beez/output_clustalo/output_tree.txt 

Hierboven is de code gerunt met de guide tree optie erbij, ook hebben wij gekozen voor 80 threads zodat de tool nog sneller klaar zou zijn.

### Tree visualisatie 

Om van de newick file naar een zichtbare phylogentische boom to gaan is de tool ete3 gebruikt. 

Deze tool zet de newick file format om in een zichtbare file. De output is een phylogentische boom.  

In [ ]:
from ete3 import Tree 

tree_path = "/students/2023-2024/Thema07/Sick_beez/output_clustalo/output_tree.txt" # Replace with the path to your Newick format file  
tree = Tree(tree_path)
tree.show() 

De boom die hier uit kwam zag er niet zo heel mooi uit, dit komt ook omdat de boom best veel entiteiten heeft. De namen van de virussen zijn ook best lastig en niet te begrijpen. 

### Missmatch en mutatie hotspots
Hier onder is code over de missmatch en mutatiehotspots grafiek. Hiervoor is de ALIGNIO van de BIO module gebruikt.

In [ ]:
from Bio import AlignIO 
import matplotlib.pyplot as plt

alignment = AlignIO.read("/students/2023-2024/Thema07/Sick_beez/output_clustalo/output_omega.msa.fa", "fasta") 
matches = [sum(1 for seq in alignment[:, i] if seq == alignment[0, i]) for i in range(alignment.get_alignment_length())] 
mismatches = [alignment.get_alignment_length() - m for m in matches ] 
plt.figure(figsize=(10, 5)) 
plt.plot(range(1, alignment.get_alignment_length() + 1), mismatches, color='red', label='Mismatches') 
plt.plot(range(1, alignment.get_alignment_length() + 1), matches, color='blue', label='Matches') 
plt.xlabel('Position in Alignment') 
plt.ylabel('Number of Matches/Mismatches') 
plt.title('Mutation Hotspots') 
plt.legend() 
plt.show() 

De grafiek die hieruit komt laat het aantal missmatches en matches per positie zien. Hieruit kan je opmaken waar de mutaties zitten.

In [ ]:
from Bio import AlignIO 
from Bio.Align import MultipleSeqAlignment 

# Load multiple sequence alignment 
alignment = AlignIO.read("alignment.fasta", "fasta") 

# Compute consensus sequence 
consensus = alignment.consensus() 
print(consensus) 

In [ ]:
vsearch --cluster_fast '/students/2023-2024/Thema07/Sick_beez/trimmed_fastq/filtered_reads.fastq.gz' --id 0.85 --strand both --clusters '/students/2023-2024/Thema07/Sick_beez/output_vsearch/cluster_vsearch.fasta'. --threads 40 --consout "/students/2023-2024/Thema07/Sick_beez/output_vsearch/consensus_vsearch.fasta"  

clustalo -i /students/2023-2024/Thema07/Sick_beez/output_vsearch/consensus_vsearch.fasta -o /students/2023-2024/Thema07/Sick_beez/output_clustalo/vsearch_omega.aln --threads 80 --guidetree-out /students/2023-2024/Thema07/Sick_beez/output_clustalo/vsearch_tree.txt 

In [ ]:
from Bio import AlignIO 
from collections import Counter 

# Load the alignment 
alignment = AlignIO.read("/students/2023-2024/Thema07/Sick_beez/output_clustalo/output_omega.msa.fa", "fasta") 

# Calculate the consensus sequence 
consensus_seq = "" 
for i in range(len(alignment[0])): 
column = alignment[:, i] 
counts = Counter(column) 
consensus_base = counts.most_common(1)[0][0] 
base_count = column.count(consensus_base) 
base_percentage = base_count / len(column) 
if base_percentage > 0.5 : 
    if consensus_base != '-': 
        consensus_seq += consensus_base 
    else: 
        consensus_seq += '-' 

# Write the consensus sequence to a file 
with open("/students/2023-2024/Thema07/Sick_beez/consensus/omega_consensus.fasta", "w") as output_file: 
    output_file.write(">Consensus\n") 
    output_file.write(consensus_seq + "\n") 

Deze python code kijkt per positie in de genomen welke base het meest voorkomt. Als deze base meer dan 50% van de keren voorkomt wordt hij in de consensus sequentie geplaatst. Als er meer dan 50% van de keren een gap is wordt deze verwijdert. Als niks meer dan 50% heeft wordt er een gap in de consensus sequentie geplaatst. 

## Primer3


In [ ]:
import primer3 

sequence = '' 
with open('/students/2023-2024/Thema07/Sick_beez/consensus/omega_consensus.fasta','r',encoding='UTF8') as consensus: 
    counter = 0 
for line in consensus: 
    if counter != 0: 
        sequence += line 
    else: 
        counter += 1 

sequence = sequence.replace('\n','') 
sequence = sequence.replace('-','N') 
sequence = sequence.replace('U','T') 

primers = primer3.bindings.design_primers( 
seq_args={ 
    'SEQUENCE_ID': 'DWV', 
    'SEQUENCE_TEMPLATE': sequence, 
    }, 
global_args={ 
    'PRIMER_OPT_SIZE': 20, 
    'PRIMER_PICK_INTERNAL_OLIGO': 1, 
    'PRIMER_INTERNAL_MAX_SELF_END': 8, 
    'PRIMER_MIN_SIZE': 18, 
    'PRIMER_MAX_SIZE': 25, 
    'PRIMER_OPT_TM': 60.0, 
    'PRIMER_MIN_TM': 57.0, 
    'PRIMER_MAX_TM': 63.0, 
    'PRIMER_MIN_GC': 20.0, 
    'PRIMER_MAX_GC': 80.0, 
    'PRIMER_MAX_POLY_X': 100, 
    'PRIMER_INTERNAL_MAX_POLY_X': 100, 
    'PRIMER_SALT_MONOVALENT': 50.0, 
    'PRIMER_DNA_CONC': 50.0, 
    'PRIMER_MAX_NS_ACCEPTED': 0, 
    'PRIMER_MAX_SELF_ANY': 12, 
    'PRIMER_MAX_SELF_END': 8, 
    'PRIMER_PAIR_MAX_COMPL_ANY': 12, 
    'PRIMER_PAIR_MAX_COMPL_END': 8, 
    'PRIMER_PRODUCT_SIZE_RANGE': [ 
    [75,100],[100,125],[125,150], 
    [150,175],[175,200],[200,225] 
    ], 
    })

with open('/students/2023-2024/Thema07/Sick_beez/primer3_output/omega_primer3.txt', 'w', encoding='UTF8') as primer_file: 
    for key, value in primers.items(): 
        primer_file.write(f"{key}: {value}\n") 